In [2]:
import pandas as pd
import numpy as np


coef_path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\Github\Python-Projects\astrea\logit_coefficients.csv"
coef = pd.read_csv(coef_path, delimiter=",")
coef.head()

,Variable,Coefficient
0,const,-3.289498
1,price,-0.584335
2,brand_strength,0.310713
3,quality_score,1.200612


In [33]:
data_path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\Github\Python-Projects\astrea\synthetic_choice_data.csv"
data = pd.read_csv(data_path, delimiter=",")
data.head()

,respondent_id,trip_id,product_id,price,brand_strength,quality_score,group,choice
0,0,0,13,2.911052,0.948886,4.579309,0,0
1,0,0,5,2.403951,0.785176,3.650089,0,0
2,0,0,7,8.795585,0.514234,3.080272,0,0
3,0,0,10,1.185260,0.607545,4.878339,0,1
4,0,0,12,8.491984,0.065052,4.757996,0,0


In [34]:
import statsmodels.api as sm

x = data[['price', 'brand_strength', 'quality_score']]
y = data['choice']

X = sm.add_constant(x)

model = sm.MNLogit(y, X)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.276433
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                 choice   No. Observations:                75000
Model:                        MNLogit   Df Residuals:                    74996
Method:                           MLE   Df Model:                            3
Date:                Thu, 13 Mar 2025   Pseudo R-squ.:                  0.4476
Time:                        10:00:24   Log-Likelihood:                -20732.
converged:                       True   LL-Null:                       -37530.
Covariance Type:            nonrobust   LLR p-value:                     0.000
==================================================================================
      choice=1       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -3.2895      0.054    -60.491      0.000      -3.396      -3.183
price             -0.5843      0.008    -75.681      0.000      -0.599      -0.569
brand_strength     0.3107      0.055      5.642      0.000       0.203       0.419
quality_score      1.2006      0.014     86.763      0.000       1.173       1.228
==================================================================================
"""

In [35]:
coefs = pd.DataFrame(results.params).reset_index()
coefs.columns = ['variable', 'coefficient']
coefs

,variable,coefficient
0,const,-3.289498
1,price,-0.584335
2,brand_strength,0.310713
3,quality_score,1.200612


In [43]:
np.exp( 1.200612)

3.322149456185058

In [36]:
data["const"] = 1.0

# Compute utility scores
utilities = (
data["const"] * coefs['coefficient'][0] +
data["price"] * coefs['coefficient'][1] +
data["brand_strength"] * coefs['coefficient'][2] +
data["quality_score"] * coefs['coefficient'][3]
)

# Apply softmax transformation within each respondent-trip group
data["exp_utilities"] = np.exp(utilities)
data["predicted_choice_prob"] = data.groupby(["respondent_id", "trip_id"])["exp_utilities"].transform(lambda x: x / x.sum())

In [41]:
pp = data[data["respondent_id"] == 7].sort_values(by='trip_id')
pp.groupby(["trip_id"]).agg({'predicted_choice_prob': 'sum'})

,predicted_choice_prob
trip_id,
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0
6,1.0
7,1.0
8,1.0


In [8]:
margins = results.get_margeff()
margins.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
       MNLogit Marginal Effects      
=====================================
Dep. Variable:                 choice
Method:                          dydx
At:                           overall
==================================================================================
      choice=0      dy/dx    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
price              0.0497      0.001     86.430      0.000       0.049       0.051
brand_strength    -0.0264      0.005     -5.663      0.000      -0.036      -0.017
quality_score     -0.1021      0.001   -110.853      0.000      -0.104      -0.100
----------------------------------------------------------------------------------
      choice=1      dy/dx    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
price             -0.0497      0.001    -86.430      0.000      -0.051      -0.049
brand_strength     0.0264      0.005      5.663      0.000       0.017       0.036
quality_score      0.1021      0.001    110.853      0.000       0.100       0.104
==================================================================================
"""

In [6]:
margins = results.get_margeff(at='mean')
margins.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
       MNLogit Marginal Effects      
=====================================
Dep. Variable:                 choice
Method:                          dydx
At:                              mean
==================================================================================
      choice=0      dy/dx    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
price              0.0380      0.001     70.298      0.000       0.037       0.039
brand_strength    -0.0202      0.004     -5.560      0.000      -0.027      -0.013
quality_score     -0.0781      0.001    -67.903      0.000      -0.080      -0.076
----------------------------------------------------------------------------------
      choice=1      dy/dx    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
price             -0.0380      0.001    -70.298      0.000      -0.039      -0.037
brand_strength     0.0202      0.004      5.560      0.000       0.013       0.027
quality_score      0.0781      0.001     67.903      0.000       0.076       0.080
==================================================================================
"""

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   respondent_id   75000 non-null  int64  
 1   trip_id         75000 non-null  int64  
 2   product_id      75000 non-null  int64  
 3   price           75000 non-null  float64
 4   brand_strength  75000 non-null  float64
 5   quality_score   75000 non-null  float64
 6   group           75000 non-null  int64  
 7   choice          75000 non-null  int64  
dtypes: float64(3), int64(5)
memory usage: 4.6 MB


In [4]:
data["choice"].unique()


array([0, 1], dtype=int64)

In [5]:
data["group"].unique()

array([0, 1], dtype=int64)

In [6]:
data["group"].value_counts()

group
1    38100
0    36900
Name: count, dtype: int64

In [7]:
data.groupby(['group', 'respondent_id']).size()

group  respondent_id
0      0                50
       1                50
       3                50
       4                50
       6                50
                        ..
1      1492             50
       1493             50
       1496             50
       1498             50
       1499             50
Length: 1500, dtype: int64

In [15]:
data["respondent_id"].nunique()

1500

In [8]:
data["respondent_id"].value_counts()

respondent_id
0       50
997     50
1006    50
1005    50
1004    50
        ..
497     50
496     50
495     50
494     50
1499    50
Name: count, Length: 1500, dtype: int64

In [9]:
data["trip_id"].unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [10]:
data["product_id"].unique()

array([13,  5,  7, 10, 12,  0,  3,  1, 19, 15, 18,  9, 17, 14, 16,  8,  2,
       11,  6,  4], dtype=int64)

In [11]:
data[data["group"] == 0]['respondent_id'].nunique()

738

In [12]:
data[data["group"] == 1]['respondent_id'].nunique()

762

In [16]:
738 + 762

1500

In [21]:
data[data['group'] == 1]


,respondent_id,trip_id,product_id,price,brand_strength,quality_score,group,choice
100,2,0,17,5.722808,0.097672,1.783931,1,0
101,2,0,3,6.387926,0.366362,4.637282,1,0
102,2,0,12,8.491984,0.065052,4.757996,1,0
103,2,0,15,2.650641,0.808397,4.687497,1,1
104,2,0,5,2.403951,0.785176,3.650089,1,0
...,...,...,...,...,...,...,...,...
74995,1499,9,5,2.403951,0.785176,3.650089,1,1
74996,1499,9,19,3.621062,0.440152,2.301321,1,0
74997,1499,9,18,4.887505,0.684233,1.180909,1,0
74998,1499,9,8,6.410035,0.592415,3.186841,1,0


In [22]:
data[data["respondent_id"] == 2].sort_values(by='trip_id')

,respondent_id,trip_id,product_id,price,brand_strength,quality_score,group,choice
100,2,0,17,5.722808,0.097672,1.783931,1,0
101,2,0,3,6.387926,0.366362,4.637282,1,0
102,2,0,12,8.491984,0.065052,4.757996,1,0
103,2,0,15,2.650641,0.808397,4.687497,1,1
104,2,0,5,2.403951,0.785176,3.650089,1,0
105,2,1,9,7.372653,0.046450,1.739418,1,0
106,2,1,0,4.370861,0.611853,1.488153,1,0
107,2,1,19,3.621062,0.440152,2.301321,1,0
108,2,1,2,7.587945,0.292145,1.137554,1,0
109,2,1,3,6.387926,0.366362,4.637282,1,1


In [14]:
data.groupby(['respondent_id']).agg({'choice': 'sum'}).reset_index().sort_values(by='choice', ascending=True)

,respondent_id,choice
0,0,10
1005,1005,10
1004,1004,10
1003,1003,10
1002,1002,10
...,...,...
495,495,10
494,494,10
493,493,10
502,502,10
